In [1]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
import urllib.request
import os
from pyquery import PyQuery as pq
import pickle
from selenium import webdriver
import datetime as dt
import time
import calendar
def get_homepage(i):
    url="https://etfdb.com/etf/"+all_ETF[i]
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    span=soup.find_all("a",text='Home page')
    url_homepage=span[0].get("href")
    return url_homepage
def get_the_last_day_of_month(start_date,end_date):
    ans = []
    temp = start_date - dt.timedelta(1)
    year = start_date.year
    month = start_date.month
    while (temp < end_date):
        temp = temp + dt.timedelta(calendar.monthrange(year,month)[1])
        ans.append(temp)
        month += 1
        if(month > 12):
            month = 1
            year += 1
    for i in range(len(ans)):
        if ans[i].isoweekday()>5:
            ans[i]=ans[i]+dt.timedelta(days=5-ans[i].isoweekday())
    return ans
last_day_of_month = get_the_last_day_of_month(dt.datetime(2016,1,1,0,0),dt.datetime(2018,12,31,0,0))
#dealing with 3/30 have no NAV
last_day_of_month[-10]=last_day_of_month[-10]+dt.timedelta(days=-1)

In [24]:
need_list=pd.read_excel("new_list.xlsx")
symbol_list = need_list['Symbol'].reset_index()['Symbol']
name_list = need_list['ETF Name'].reset_index()['ETF Name']
all_ETF=need_list["Symbol"].values
all_ETF

array(['EWM', 'ADRE', 'EEM', 'FXI', 'PGJ', 'VWO', 'EEV', 'EUM', 'SPEM',
       'GXC', 'GMF', 'FXP', 'DEM', 'DGS', 'FNI', 'AIA', 'PIE', 'HAO',
       'EPI', 'THD', 'PIN', 'AAXJ', 'EWX', 'EDC', 'EDZ', 'IDX', 'EET',
       'XPP', 'YINN', 'YANG', 'CQQQ', 'CHIC', 'CHIX', 'YAO', 'VNM', 'INDY',
       'CHII', 'CHIQ', 'CHIE', 'YXI', 'EIDO', 'SCIN', 'SCIF', 'ECNS',
       'EPHE', 'PEK', 'INDL', 'INXX', 'CHIM', 'ASEA', 'EDIV', 'MCHI',
       'FEM', 'FCA', 'DBEM', 'INCO', 'EEMV', 'INDA', 'NFTY', 'FEMS',
       'SMIN', 'EEMA', 'CXSE', 'DGRE', 'ASHR', 'KFYP', 'KWEB', 'AXJV',
       'KBA', 'CN', 'ASHS', 'CNXT', 'EMQQ', 'XCEM', 'CHAU', 'CHAD', 'EMGF',
       'ASHX', 'CNHX', 'XINA', 'KALL', 'AFTY'], dtype=object)

In [ ]:
with open('ETF_dic.pkl','rb') as f:
    dic = pickle.load(f)

In [22]:
def check_download_file(com_data):
    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': './data_yahoo'}
    options.add_experimental_option('prefs', prefs)
    browser=webdriver.Chrome(options=options) 
    
    for s in dic['symbol']:
        data = com_data[s].values
        if (np.sum(np.isnan(data)) > 10):
            os.remove(f'./data_yahoo/{s}.csv')
            get_single_symbol(s,browser,5)
    browser.quit()
def get_single_symbol(symbol,browser,wait = 3):
    
    # check directory
    if not os.path.isdir('./data_yahoo'):
        os.mkdir('./data_yahoo')
    
    filepath = f'./data_yahoo/{symbol}.csv'
    if os.path.isfile(filepath):
        print('File exist')
        return 
    
    url = f'https://finance.yahoo.com/quote/{symbol}/history?p={symbol}'
    browser.get(url)
    browser.find_element_by_css_selector("[class='C(t) O(n):f Tsh($actionBlueTextShadow) Bd(n) Bgc(t) Fz(14px) Pos(r) T(-1px) Bd(n):f Bxsh(n):f Cur(p) W(190px)']").click()
    start_date = browser.find_element_by_css_selector("[class='Bdrs(0) Bxsh(n)! Fz(s) Bxz(bb) D(ib) Bg(n) Pend(5px) Px(8px) Py(0) H(34px) Lh(34px) Bd O(n):f O(n):h Bdc($c-fuji-grey-c) Bdc($c-fuji-blue-1-b):f M(0) Pstart(10px) Bgc(white) W(90px) Mt(5px)']")
    start_date.clear()
    start_date.send_keys('1/1/2016')
    end_date = browser.find_element_by_css_selector("[class='Bdrs(0) Bxsh(n)! Fz(s) Bxz(bb) D(ib) Bg(n) Pend(5px) Px(8px) Py(0) H(34px) Lh(34px) Bd O(n):f O(n):h Bdc($c-fuji-grey-c) Bdc($c-fuji-blue-1-b):f M(0) Pstart(10px) Bgc(white) W(90px) Mt(5px) Mstart(15px)']")
    end_date.clear()
    end_date.send_keys('1/1/2019')

    try:
        browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Miw(80px)! Fl(start)']").click()
    except:
        browser.find_element_by_css_selector("[class=' Bd Bdc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) D(ib) C($c-fuji-blue-1-b) Bdc($actionBlueHover):h C($actionBlueHover):h Cur(p) Td(n)  Py(8px) cancel Miw(80px)! Fl(end)']").click()
        browser.find_element_by_css_selector("[class='C(t) O(n):f Tsh($actionBlueTextShadow) Bd(n) Bgc(t) Fz(14px) Pos(r) T(-1px) Bd(n):f Bxsh(n):f Cur(p) W(190px)']").click()
        browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Miw(80px)! Fl(start)']").click()

    browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Fl(end)']").click()
    time.sleep(5)
    browser.find_element_by_css_selector("[class='Fl(end) Mt(3px) Cur(p)']").click()
    time.sleep(wait)

    while not os.path.isfile(filepath):
        browser.find_element_by_css_selector("[class='Fl(end) Mt(3px) Cur(p)']").click()
        time.sleep(wait)

    print('Download',symbol)
def get_all_symbol_csv(dic):
    
    # check directory
    if not os.path.isdir('./data_yahoo'):
        os.mkdir('./data_yahoo')

    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': './data_yahoo'}
    options.add_experimental_option('prefs', prefs)
    browser=webdriver.Chrome(options=options) 
    
    for s in dic['symbol']:
        get_single_symbol(s,browser)

    browser.quit()
def combine_data(dic,last_day_of_month):
    com_data = pd.DataFrame(last_day_of_month[::-1],columns=['Date'])
        
    for s in dic['symbol']:
        data = pd.read_csv(f'./data_yahoo/{s}.csv')
        data['Date'] = pd.to_datetime(data['Date'], format="%Y-%m-%d")
        
        # Mask
        mask = [False for i in range(data['Date'].shape[0])]
        for day in last_day_of_month:
            temp = data['Date'] == day
            mask |= temp
        data = data[mask]
        
        data_close = data[['Date','Close']].reset_index(drop=True)
        data_close = data_close.rename(columns={'Close':s})
        com_data = com_data.merge(data_close,on='Date',how='left')
        
    # check directory
    if not os.path.isdir('./Combine_data'):
        os.mkdir('./Combine_data')
    com_data.to_csv('./Combine_data/com_data.csv')
    
    return com_data
def check_download_file(com_data):
    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': './data_yahoo'}
    options.add_experimental_option('prefs', prefs)
    browser=webdriver.Chrome(options=options) 
    
    for s in dic['symbol']:
        data = com_data[s].values
        if (np.sum(np.isnan(data)) > 10):
            os.remove(f'./data_yahoo/{s}.csv')
            get_single_symbol(s,browser,5)
    browser.quit()

In [30]:
#get yahoo_finance_data
get_all_symbol_csv(dic)

File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist


,EWM,ADRE,EEM,FXI,PGJ,VWO,EEV,EUM,SPEM,GXC,...,EMQQ,XCEM,CHAU,CHAD,EMGF,ASHX,CNHX,XINA,KALL,AFTY
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-29,32.439999,27.000000,30.570000,31.200001,28.840000,30.830000,129.199997,30.719999,24.465000,64.110001,...,20.966999,18.184000,14.600000,50.130001,31.240000,20.040001,23.510000,18.870001,NaN,11.790000
2016-02-29,31.320000,27.290001,30.320000,30.280001,27.959999,30.730000,128.649994,30.790001,24.455000,63.209999,...,20.180000,18.639999,14.369000,49.220001,31.250000,19.650000,24.250000,19.510000,NaN,11.700000
2016-03-31,35.639999,30.969999,34.250000,33.770000,30.020000,34.580002,99.199997,27.080000,27.360001,69.790001,...,22.450001,21.190001,18.459999,43.139999,35.139999,21.850000,26.120001,21.190001,NaN,12.990000
2016-04-29,34.200001,31.520000,34.389999,33.529999,30.459999,34.930000,97.199997,26.860001,27.740000,69.410004,...,22.790001,21.780001,17.450001,44.110001,35.500000,21.309999,25.549999,21.180000,NaN,12.760000
2016-05-31,31.680000,30.440001,33.119999,33.650002,29.600000,33.799999,103.800003,27.770000,26.875000,69.349998,...,22.969999,20.420000,17.809999,43.439999,34.240002,21.799999,24.219999,20.966999,NaN,12.730000
2016-06-30,33.160000,31.270000,34.360001,34.220001,28.250000,35.230000,92.300003,26.290001,27.780001,69.889999,...,22.521999,21.950001,17.209999,43.700001,35.419998,21.059999,24.730000,21.090000,NaN,12.360000
2016-07-29,32.840000,32.830002,36.209999,35.430000,29.719999,37.040001,82.599998,24.910000,29.125000,72.080002,...,23.590000,22.945000,18.020000,42.380001,37.990002,22.392000,26.330000,22.100000,NaN,13.120000
2016-08-31,32.720001,34.009998,36.529999,37.049999,31.620001,37.330002,80.599998,24.620001,29.674999,76.699997,...,25.860001,23.459999,19.320000,40.810001,37.480000,23.000000,26.330000,22.100000,NaN,13.500000
2016-09-30,32.439999,34.840000,37.450001,38.009998,33.180000,37.630001,75.349998,23.850000,30.290001,79.910004,...,27.259001,23.552000,18.469999,41.540001,38.639999,22.379999,26.370001,21.655001,NaN,13.520000


In [47]:
#combine_all_data
com_data = combine_data(dic,last_day_of_month)
com_data.index=com_data["Date"]
com_data=com_data.iloc[::-1,1:]

In [2]:
#將太晚發行ETF拿掉
"""
origin_list=pd.read_excel("origin_list.xlsm",index_col=22,parse_dates=True)
need_list=origin_list.sort_index().loc[:"2016-01-01"]
need_list.to_excel("new_list.xlsx")
"""

In [133]:
"""
homepage_list = []
for i in symbol_list:
    url = 'https://www.etf.com/'+i+'#overview'
    res = requests.get(url)
    doc = pq(res.text)
    doc.make_links_absolute(res.url)
    homepage_list.append(doc('.field-content.helplink')('a').attr('href'))
dic = {'symbol':[],'name':[],'addr':[]}
for i,s in enumerate(symbol_list):
    dic['symbol'].append(s)
    dic['name'].append(name_list[i])
    dic['addr'].append(homepage_list[i])
    print(s,name_list[i],homepage_list[i])
    
 # Save this dic
with open("ETF_dic.pkl","wb") as f:
    pickle.dump(dic,f)
"""

'\nhomepage_list = []\nfor i in symbol_list:\n    url = \'https://www.etf.com/\'+i+\'#overview\'\n    res = requests.get(url)\n    doc = pq(res.text)\n    doc.make_links_absolute(res.url)\n    homepage_list.append(doc(\'.field-content.helplink\')(\'a\').attr(\'href\'))\ndic = {\'symbol\':[],\'name\':[],\'addr\':[]}\nfor i,s in enumerate(symbol_list):\n    dic[\'symbol\'].append(s)\n    dic[\'name\'].append(name_list[i])\n    dic[\'addr\'].append(homepage_list[i])\n    print(s,name_list[i],homepage_list[i])\n    \n # Save this dic\nwith open("ETF_dic.pkl","wb") as f:\n    pickle.dump(dic,f)\n'

In [44]:
#print all ETF address
for i in range(len(all_ETF)):
    try:
        url="https://etfdb.com/etf/"+all_ETF[i]
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, 'html.parser')
        span=soup.find_all("a",text='Home page')
        url_homepage=span[0].get("href")
        print (url_homepage)
    except:
        print(all_ETF[i])     

http://us.ishares.com/product_info/fund/overview/EWM.htm?qt=EWM
https://www.invesco.com/portal/site/us/financial-professional/etfs/product-detail?productId=ADRE
http://us.ishares.com/product_info/fund/overview/EEM.htm?qt=EEM
http://us.ishares.com/product_info/fund/overview/FXI.htm?qt=FXI
http://invescopowershares.com/products/overview.aspx?ticker=PGJ
https://personal.vanguard.com/us/funds/snapshot?FundId=0964&FundIntExt=INT
http://www.proshares.com/funds/eev.html
http://www.proshares.com/funds/eum.html
https://us.spdrs.com/en/etf/spdr-portfolio-emerging-markets-etf-SPEM
https://www.spdrs.com/product/fund.seam?ticker=GXC
https://www.spdrs.com/product/fund.seam?ticker=GMF
http://www.proshares.com/funds/fxp.html
http://www.wisdomtree.com/etfs/fund-details.asp?etfid=50
http://www.wisdomtree.com/etfs/fund-details.asp?etfid=53
http://www.ftportfolios.com/retail/ETF/ETFsummary.aspx?Ticker=FNI
http://us.ishares.com/product_info/fund/overview/AIA.htm?qt=AIA
http://invescopowershares.com/product

In [33]:
# print unsolve website
for i in range(len(all_ETF)):
    try:
        url="https://etfdb.com/etf/"+all_ETF[i]
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, 'html.parser')
        span=soup.find_all("a",text='Home page')
        url_homepage=span[0].get("href")
        if "ishares" in url_homepage:#處理ishare的ETF
            continue
        elif "invesco" in url_homepage:#處理invesco的ETF
            continue
        elif "vanguard" in url_homepage: #vanguard
            continue
        elif "proshares" in url_homepage: #proshares
            continue
        elif "spdrs" in url_homepage: #spdrs
            continue
        elif "wisdomtree" in url_homepage: #wisdomtree
            continue
        elif "vaneck" in url_homepage:#vaneck
            continue
        elif "emerging" in url_homepage:#emerging
            continue
        elif "globalxfunds" in url_homepage:#globalxfunds
            continue
        else:
            print (url_homepage)
    except:
        print(all_ETF[i])
        

http://www.ftportfolios.com/retail/ETF/ETFsummary.aspx?Ticker=FNI
http://www.guggenheimfunds.com/etf/fund/HAO
http://direxionshares.com/etf/embu_bull_3x_shares.html
http://direxionshares.com/etf/embe_bear_3x_shares.html
http://www.direxioninvestments.com/products/direxion-daily-china-bull-3x-etf
http://www.direxioninvestments.com/products/direxion-daily-china-bull-3x-etf
http://guggenheiminvestments.com/products/etf/cqqq
CHIC
http://guggenheiminvestments.com/products/etf/yao
http://www.direxionshares.com/etf/indiabu_2x_shares.html
http://www.ftportfolios.com/Retail/etf/etfsummary.aspx?Ticker=FEM
http://www.ftportfolios.com/Retail/etf/etfsummary.aspx?Ticker=FCA
https://etfus.deutscheawm.com/US/EN/Product-Detail-Page/DBEM
NFTY
http://www.ftportfolios.com/retail/etf/etfsummary.aspx?Ticker=FEMS
https://etfus.deutscheawm.com/US/EN/Product-Detail-Page/ASHR
http://kraneshares.com/kfyp/
http://kraneshares.com/kweb/
http://kraneshares.com/kba/
https://etfus.deutscheawm.com/US/EN/Product-Detail-

In [ ]:
#ishare 未能讀檔 0
def download_ishare(i,url_homepage):
    resp = requests.get(url_homepage)
    soup = BeautifulSoup(resp.text, 'html.parser')
    excel_file=soup.find_all("a",{"class":"icon-xls-export"})[3].get("href")
    domain_url = "https://www.ishares.com"
    excel_file= domain_url+excel_file
    file_name="data/"+all_ETF[i]+".xls"
    urllib.request.urlretrieve(excel_file, file_name)
#invesco 1
def download_invesco(i,url_homepage):
    options = webdriver.ChromeOptions()
    prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': '/Users/shayumin/Documents/文字探勘/homework1/data'}
    options.add_experimental_option('prefs', prefs)
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url_homepage)
    driver.find_element_by_link_text("Historical NAVs").click()
    time.sleep(5)
    driver.quit()
    testdir = '/Users/shayumin/Documents/文字探勘/homework1/data'
    list = os.listdir(testdir)
    list.sort(key=lambda fn:os.path.getmtime(testdir+'/'+fn))
    file_name=all_ETF[i]+".csv"
    os.rename(os.path.join(testdir,list[-1]),os.path.join(testdir,file_name))
    NAV=pd.read_csv("data/"+all_ETF[i]+".csv",index_col=2,parse_dates=True).loc[last_day_of_month]["NAV"]
    return NAV
#vanguard 5
def get_table(search_url,beginDate,endDate):
    driver = webdriver.Chrome()
    driver.get(search_url)
    driver.find_element_by_name("beginDate").send_keys(beginDate)
    driver.find_element_by_name("endDate").send_keys(endDate)
    driver.find_element_by_id("_cbdExt8").click()
    page_content=pd.read_html(driver.current_url)
    data_table=page_content[1].iloc[54:,:2]
    data_table=pd.DataFrame(data_table[1].values,index=pd.to_datetime(data_table[0]),columns=["Nav"])
    driver.quit()
    return  data_table
def download_vanguard(i,url_homepage):
    url_homepage=get_homepage(i)
    search_url="https://personal.vanguard.com/us/funds/tools/pricehistorysearch?"+url_homepage[url_homepage.find("FundId"):url_homepage.find("FundIntExt")-1]
    data_table_2016=get_table(search_url,"01/01/2016","12/31/2016")
    data_table_2017=get_table(search_url,"01/01/2017","12/31/2017")
    data_table_2018=get_table(search_url,"01/01/2018","12/31/2018")
    NAV=pd.concat([data_table_2016,data_table_2017,data_table_2018])
    NAV=NAV.loc[last_day_of_month]
    return NAV["Nav"]
#proshares 6
def download_proshares(i,url_homepage):
    options = webdriver.ChromeOptions()
    prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': '/Users/shayumin/Documents/文字探勘/homework1/data'}
    options.add_experimental_option('prefs', prefs)
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url_homepage)
    driver.find_element_by_link_text("NAV History").click()
    time.sleep(10)
    driver.quit()
    testdir = '/Users/shayumin/Documents/文字探勘/homework1/data'
    list = os.listdir(testdir)
    list.sort(key=lambda fn:os.path.getmtime(testdir+'/'+fn))
    filename=all_ETF[i]+'.csv'
    os.rename(os.path.join(testdir,list[-1]),os.path.join(testdir,filename))
    NAV=pd.read_csv("data/"+all_ETF[i]+'.csv',index_col=0,parse_dates=True).loc[last_day_of_month]["NAV"]
    return NAV
#spdrs 9
def download_spdrs(i,url_homepage):
    domain_url = "https://us.spdrs.com/"
    excel_file= domain_url+"site-content/xls/"+all_ETF[9]+"_HistoricalNav.xls?fund="+all_ETF[9]+"&amp;docname=Most+Recent+NAV+%2F+NAV+History&amp;onyx_code1=&amp;onyx_code2="
    excel_file
    file_name="data/"+all_ETF[i]+".xls"
    urllib.request.urlretrieve(excel_file, file_name)
    file=pd.read_excel(file_name,index_col=0,header=2,parse_dates=True).dropna(axis=1,how="all")[:-12]
    file.index=pd.to_datetime(file.index)
    Nav=file["Nav"].loc[last_day_of_month]
    return(Nav)
#globalxfunds 32
def download_globalxfunds(i,url_homepage):
    ETF_len=len(all_ETF[i])
    download_link=url_homepage[:-ETF_len]+"fund/"+url_homepage[-ETF_len:]+"/?download_chart_data=true"
    file_name="data/"+all_ETF[i]+".csv"
    urllib.request.urlretrieve(download_link, file_name)
    NAV=pd.read_csv(file_name,index_col=0,header=1,parse_dates=True)["Closing Price"].loc[last_day_of_month]
    return NAV
#emerging 41
def download_emerging(i,url_homepage):
    options = webdriver.ChromeOptions()
    prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': '/Users/shayumin/Documents/文字探勘/homework1/data'}
    options.add_experimental_option('prefs', prefs)
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url_homepage)
    driver.find_element_by_link_text("Download NAV/Premium Discount").click()
    time.sleep(40)
    driver.quit()
    testdir = '/Users/shayumin/Documents/文字探勘/homework1/data'
    list = os.listdir(testdir)
    list.sort(key=lambda fn:os.path.getmtime(testdir+'/'+fn))
    file_name=all_ETF[i]+".xls"
    os.rename(os.path.join(testdir,list[-1]),os.path.join(testdir,file_name))
    path="data/"+file_name
    day_NAV=pd.read_excel(path,index_col=0,header=4,parse_dates=True).loc[last_day_of_month]
    NAV=day_NAV[day_NAV["Fund Symbol"]==all_ETF[i]]["NAV"]
    return NAV

In [ ]:
for i in range(1):
    try:
        url_homepage=get_homepage(i)
        print (url_homepage)
#        if "ishares" in url_homepage:#處理ishare的ETF
#            download_ishare(i,url_homepage)
        if "invesco" in url_homepage:#處理invesco的ETF
            NAV=download_invesco(i,url_homepage)
            com_data[all_ETF[i]]=NAV
        if "vanguard" in url_homepage: #vanguard
            NAV=download_vanguard(i,url_homepage)
            com_data[all_ETF[i]]=NAV
        if "proshares" in url_homepage: #proshares
            NAV=download_proshares(i,url_homepage)
            com_data[all_ETF[i]]=NAV
        if "spdrs" in url_homepage: #spdrs
            NAV=download_spdrs(i,url_homepage)
            com_data[all_ETF[i]]=NAV
        if "wisdomtree" in url_homepage: #wisdomtree
            url="https://etfdb.com/etf/"+all_ETF[13]
            resp = requests.get(url)
            soup = BeautifulSoup(resp.text, 'html.parser')
            span=soup.find_all("a",text='Home page')
            url_homepage=span[0].get("href")
            resp = requests.get(url_homepage)
            soup = BeautifulSoup(resp.text, 'html.parser')
            link_address=soup.find_all("a",{"class":"text-link fund-modal-trigger"})[0].get("data-href")
            data=pd.read_html(link_address,index_col=0,parse_dates=True)
        if "vaneck" in url_homepage:#vaneck
        if "emerging" in url_homepage:#emerging
            NAV=download_emerging(i,url_homepage)
            com_data[all_ETF[i]]=NAV
        if "globalxfunds" in url_homepage:#globalxfunds
            NAV=download_globalxfunds(i,url_homepage)
            com_data[all_ETF[i]]=NAV
    except:
        print(all_ETF[i])

In [101]:
#wisdomtree
url="https://etfdb.com/etf/"+all_ETF[13]
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html.parser')
span=soup.find_all("a",text='Home page')
url_homepage=span[0].get("href")
resp = requests.get(url_homepage)
soup = BeautifulSoup(resp.text, 'html.parser')
link_address=soup.find_all("a",{"class":"text-link fund-modal-trigger"})[0].get("data-href")
data=pd.read_html(link_address,index_col=0,parse_dates=True)

In [102]:
data

[                Nav  Mid Bid/Ask P/D to NAV P/D Indicator
 Date                                                     
 2019-03-14  46.6799       46.695      0.03%       Premium
 2019-03-13  46.7840       47.005      0.47%       Premium
 2019-03-12  46.6998       46.830      0.28%       Premium
 2019-03-11  46.3015       46.720      0.90%       Premium
 2019-03-08  46.0165       46.015      0.00%      Discount
 2019-03-07  46.4036       46.100     -0.66%      Discount
 2019-03-06  46.8761       46.855     -0.05%      Discount
 2019-03-05  46.8414       47.105      0.56%       Premium
 2019-03-04  46.8092       46.685     -0.27%      Discount
 2019-03-01  46.6440       46.550     -0.20%      Discount
 2019-02-28  46.5136       46.345     -0.36%      Discount
 2019-02-27  46.8602       46.860      0.00%      Discount
 2019-02-26  46.9401       47.205      0.56%       Premium
 2019-02-25  47.1221       47.320      0.42%       Premium
 2019-02-22  46.6208       46.815      0.42%       Premi

In [100]:
get_homepage(13)

'http://www.wisdomtree.com/etfs/fund-details.asp?etfid=53'

In [9]:
#vaneck
url="https://etfdb.com/etf/"+all_ETF[25]
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html.parser')
span=soup.find_all("a",text='Home page')
url_homepage=span[0].get("href")
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': '/Users/shayumin/Documents/文字探勘/homework1'}
options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(chrome_options=options)
url_link="https://www.vaneck.com/etf/equity/idx/documents/"
driver.get(url_homepage)
try:
    time.sleep(1)
    driver.find_element_by_link_text("Individual Investor").click()
    driver.find_element_by_link_text("I agree").click()
except:
    time.sleep(1)
time.sleep(3)
driver.find_element_by_link_text("Documents").click()

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"Documents"}
  (Session info: chrome=72.0.3626.121)
  (Driver info: chromedriver=2.46.628411 (3324f4c8be9ff2f70a05a30ebc72ffb013e1a71e),platform=Mac OS X 10.13.6 x86_64)
